## Day 23 - longest path down

**Part 1: Find longest path from single top row point to single bottom row point**

- `>` etc. tiles means you have to go that direction no other choice (stepe slope)

In [3]:
with open("./example.txt") as f:
    example_grid = [[*line.strip()] for line in f.readlines()]

with open("./input.txt") as f:
    input_grid = [[*line.strip()] for line in f.readlines()]

In [4]:
def update_been_there_dict(been_there: dict[tuple, set], old_id: int, new_id: int, base_path_new_to_ignore: tuple[int, int]):
    for k, v in been_there.items():
        if k == base_path_new_to_ignore:
            continue
        if old_id in v:
            been_there[k].add(new_id)

In [5]:
len(example_grid[0])

23

In [6]:
def part1(grid: list[list[str]]) -> int:
    """
    Finding longest possible path from top to bottom.
    """
    # using (y,x) coords with top left 0,0 s.t. access grid[y][x]
    sy, sx = (0, grid[0].index("."))
    ending_pos = (len(grid) - 1, grid[-1].index("."))
    # both input and example have these in (0, 1) and (-1, -2) but this generalises I suppose!

    # (steps, y, x, pathID)
    # initially no steps, at the starting position
    initial_pathID = 1
    # initial = (1e10, sy, sx, initial_pathID)
    initial = (-1, sy, sx, initial_pathID)

    paths = [initial]
    been_there = {
        (sy, sx): {initial_pathID}
    }

    answers = set()
    used_IDs = {1}

    while paths:
        steps, y, x, path_id = paths.pop()

        steps += 1

        if (y, x) == ending_pos:
            answers.add(abs(steps))

        if grid[y][x] == ".":
            # any valid direction
            first = True
            base_path_id = path_id
            for dy, dx in ((1, 0), (-1, 0), (0, 1), (0, -1)):
                ny = y + dy
                nx = x + dx
                if (0 <= ny <= len(grid) - 1) and (0 <= nx <= len(grid[0]) - 1) and path_id not in (been_value := been_there.get((ny, nx), set())):
                    if grid[ny][nx] != "#":
                        new_path_id = path_id
                        if first:
                            first = False
                            base_path_new_to_ignore = (ny, nx)
                        else:
                            # messy way of getting new paths onto new unique ids
                            while True:
                                new_path_id = new_path_id + 1
                                if new_path_id not in used_IDs:
                                    used_IDs.add(new_path_id)
                                    break
                            update_been_there_dict(been_there, base_path_id, new_path_id, base_path_new_to_ignore)
                        # we move
                        paths.append((steps, ny, nx, new_path_id))
                        been_value.add(new_path_id)
                        been_there[(ny, nx)] = been_value
            continue

        if grid[y][x] == ">":
            # go right
            ny = y + 0
            nx = x + 1
        elif grid[y][x] == "v":
            # go down
            ny = y + 1
            nx = x + 0
        elif grid[y][x] == "<":
            # go left  -  also never happens in my input
            ny = y + 0
            nx = x - 1
        elif grid[y][x] == "^":
            # go up  -  never happens in my input
            ny = y - 1
            nx = x + 0
        else:
            raise RuntimeError("Invalid tile!")
        
        if grid[ny][nx] != "#" and path_id not in (been_value := been_there.get((ny, nx), set())):
            paths.append((steps, ny, nx, path_id))
            been_value.add(path_id)
            been_there[(ny, nx)] = been_value

    return max(answers)

assert part1(example_grid) == 94
part1(input_grid)  # a little slow but we're ok with it

2202

**Part 2: The same without steep slopes**

In [7]:
def kill_irrelevant_ids(been_there: dict, dead_id):
    for k, v in been_there.items():
        if dead_id in v:
            been_there[k].remove(dead_id)

In [14]:
import sys
def part2(grid: list[list[str]]) -> int:
    """
    Finding longest possible path from top to bottom. But now it's more complicated bc no steep sloped (higher complexity!)
    """
    # using (y,x) coords with top left 0,0 s.t. access grid[y][x]
    sy, sx = (0, grid[0].index("."))
    ending_pos = (len(grid) - 1, grid[-1].index("."))

    # (steps, y, x, pathID)
    # initially no steps, at the starting position
    initial_pathID = 1
    # initial = (1e10, sy, sx, initial_pathID)
    initial = (-1, sy, sx, initial_pathID)

    paths = [initial]
    been_there = {
        (sy, sx): {initial_pathID}
    }

    answer = 0
    used_IDs = {1}

    while paths:
        current = paths.pop()
        current_path = [current]
        while current_path:
            steps, y, x, path_id = current_path.pop()
            steps += 1

            if (y, x) == ending_pos:
                prev_print = answer
                answer = max(answer, steps)
                if answer != prev_print:
                    print(f"New one! {answer=}")
                    print("paths size", sys.getsizeof(paths))
                    print("been there size", sys.getsizeof(been_there))
                current_path = []
                kill_irrelevant_ids(been_there, path_id)
                break

            # any valid direction
            first = True
            base_path_id = path_id
            new_path_id = path_id
            for dy, dx in ((1, 0), (-1, 0), (0, 1), (0, -1)):
                ny = y + dy
                nx = x + dx
                if (0 <= ny <= len(grid) - 1) and (0 <= nx <= len(grid[0]) - 1) and path_id not in (been_value := been_there.get((ny, nx), set())):
                    if grid[ny][nx] != "#":
                        if first:
                            current_path.append(
                                (steps, ny, nx, new_path_id)
                            )
                            first = False
                            been_value.add(new_path_id)
                            been_there[(ny, nx)] = been_value
                            base_path_new_to_ignore = (ny, nx)
                        else:
                            # messy way of getting new paths onto new unique ids
                            while True:
                                new_path_id = new_path_id + 1
                                if new_path_id not in used_IDs:
                                    used_IDs.add(new_path_id)
                                    break
                            update_been_there_dict(been_there, base_path_id, new_path_id, base_path_new_to_ignore)
                            # we move
                            paths.append((steps, ny, nx, new_path_id))
                            been_value.add(new_path_id)
                            been_there[(ny, nx)] = been_value

    return answer

assert part2(example_grid) == 154
part2(input_grid) # 6222 not right (i think too small)

New one! answer=154
paths size 120
been there size 4688
New one! answer=5102
paths size 472
been there size 147544
New one! answer=5294
paths size 472
been there size 294992
New one! answer=5494
paths size 472
been there size 294992
New one! answer=5522
paths size 472
been there size 294992
New one! answer=5602
paths size 472
been there size 294992
New one! answer=5762
paths size 472
been there size 294992
New one! answer=5842
paths size 472
been there size 294992
New one! answer=5990
paths size 472
been there size 294992
New one! answer=6014
paths size 472
been there size 294992
New one! answer=6034
paths size 472
been there size 294992


KeyboardInterrupt: 